In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
PATH='/data/notebook/'

cicids_competitors=open(PATH+"cicids_competitors.csv", "a")

In [ ]:
#!{sys.executable} -m pip install pyod
#!{sys.executable} -m pip install tgan

In [ ]:
import tgan
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import IsolationForest
from pyod.models.cblof import CBLOF
from tgan.model import TGANModel
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef
pd.set_option('display.max_colwidth', None)
MKL_NUM_THREADS=6
OMP_NUM_THREADS=6

In [ ]:
# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))

    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)

    # make the distribution fit [0; 1] by dividing by its range
    if(value_range!=0):
        return starts_from_zero / value_range
    else:
        return starts_from_zero

def compute_TSNE(perplexity,X_data):
    tsne =TSNE(n_components=2, perplexity=perplexity,
     early_exaggeration=12.0, learning_rate='auto',
     n_iter=1000, n_iter_without_progress=300,
     min_grad_norm=1e-07, metric='euclidean',#init='pca',
     verbose=0, 
     method='barnes_hut', angle=0.5, n_jobs=12).fit_transform(X_data)
    
    
    # extract x and y coordinates representing the positions of the images on T-SNE plot
    tx = tsne[:, 0]
    ty = tsne[:, 1]

    tx = scale_to_01_range(tx)
    ty = scale_to_01_range(ty)
    return tx, ty

In [ ]:
CICIDS_train_X=np.load('/home/notebook/cicids/CICIDS_train_X_no_attack.npy')[:,0:77]
CICIDS_train_Y=np.zeros((CICIDS_train_X.shape[0],1))
CICIDS_test_X=np.load('/home/notebook/cicids/CICIDS_test_X.npy')[:, 0:77]
CICIDS_test_Y=np.load('/home/notebook/cicids/CICIDS_test_Y.npy')

In [ ]:
contamination= (np.sum(CICIDS_train_Y, axis=0)/CICIDS_train_Y.shape[0])[0]


In [ ]:
pd.set_option('display.max_columns', 500)


In [ ]:
CICIDS_columns=pd.read_csv("./datasets/CICIDS18_Shuffled_Reduced.csv")
CICIDS_columns

In [ ]:
CICIDS_columns=CICIDS_columns.drop(columns=['label'])
CICIDS_columns=CICIDS_columns.drop(columns=['Timestamp'])

CICIDS_train_X=pd.DataFrame(CICIDS_train_X, columns=list(CICIDS_columns.columns.values))
CICIDS_train_Y=pd.DataFrame(CICIDS_train_Y, columns=['label'])
CICIDS_test_X=pd.DataFrame(CICIDS_test_X, columns=list(CICIDS_columns.columns.values))
CICIDS_test_Y=pd.DataFrame(CICIDS_test_Y, columns=['label'])
CICIDS_train_X

In [ ]:
print("train size, test size, contamination in train")
CICIDS_train_X_no_attack=pd.concat([CICIDS_train_X, CICIDS_train_Y], axis=1)
CICIDS_train_X_no_attack=CICIDS_train_X_no_attack[CICIDS_train_X_no_attack.label == 0]
CICIDS_train_X_no_attack=CICIDS_train_X_no_attack.drop(['label'], axis=1)
CICIDS_train_X_no_attack

In [ ]:
#pd.set_option('display.max_rows', 500)
#CICIDS_X.dtypes

In [ ]:
discrete_columns = ['Protocol','Fwd_PSH_Flags','Bwd_PSH_Flags','Fwd_URG_Flags', 'Bwd_URG_Flags',
                    'FIN_Flag_Cnt','SYN_Flag_Cnt','RST_Flag_Cnt','PSH_Flag_Cnt','ACK_Flag_Cnt',
                    'URG_Flag_Cnt','CWE_Flag_Count','ECE_Flag_Cnt','DownUp_Ratio','Fwd_Bytsb_Avg',
                    'Fwd_Pktsb_Avg','Fwd_Blk_Rate_Avg','Bwd_Bytsb_Avg','Bwd_Pktsb_Avg','Bwd_Blk_Rate_Avg']

all_columns=list(CICIDS_train_X_no_attack.columns.values)

In [ ]:
all_columns

In [ ]:
continuous_columns= list(set(all_columns) - set(discrete_columns))
continuous_columns

In [ ]:
continuous_columns=[CICIDS_train_X_no_attack.columns.get_loc(c) for c in continuous_columns if c in CICIDS_train_X_no_attack]

In [ ]:
tgan = TGANModel(continuous_columns,
                 max_epoch=20,#10
                 steps_per_epoch=100,
                 save_checkpoints=True,
                 restore_session=False,
                 batch_size=1024,
                 z_dim=100,
                 noise=0.2,
                 l2norm=0.0001,
                 learning_rate=0.001,
                 num_gen_rnn=100,
                 num_gen_feature=100,
                 num_dis_layers=1,
                 num_dis_hidden=100,
                 optimizer='AdamOptimizer'
                )

tgan.fit(CICIDS_train_X_no_attack)

In [ ]:
num_samples = 50000
X_ood = tgan.sample(num_samples)

In [ ]:
X_ood=X_ood.apply(pd.to_numeric)

In [ ]:
#remove identical elements from the generated attacks
a= CICIDS_train_X_no_attack.to_numpy()
b=X_ood.to_numpy()
res = (b[:, None] == a).all(-1).any(-1)
c=b[np.array(res==False)]
X_ood=c
CICIDS_train_X_no_attack.to_numpy()
CICIDS_train_X_no_attack.shape, X_ood.shape

In [ ]:
from scipy.spatial import distance
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
CICIDS_train_X__no_attacks_augmented=np.vstack((CICIDS_train_X_no_attack, X_ood))
CICIDS_train_Y__no_attacks_augmented=np.vstack((np.zeros((CICIDS_train_X_no_attack.shape[0],1)),
                                                 np.ones((X_ood.shape[0], 1))))

In [ ]:
xb=xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False )
xb.fit(CICIDS_train_X__no_attacks_augmented, CICIDS_train_Y__no_attacks_augmented)
predicted=xb.predict(CICIDS_test_X)

accuracy=accuracy_score(CICIDS_test_Y, predicted)
tn, fp, fn, tp = confusion_matrix(CICIDS_test_Y, predicted).ravel()
mcc=matthews_corrcoef(CICIDS_test_Y, predicted)
print("tp, tn, fp, fn, accuracy, mcc:")
tp, tn, fp, fn, accuracy, mcc

In [ ]:
dataset='cicids'
normal_in_training_set=np.unique(CICIDS_train_X_no_attack.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]
generated_attacks=X_ood.shape[0]
cicids_competitors.write(dataset+', '+
                       'TabGAN, '+
                       'XGBoost, '+
                       '0.6--0.4, '+
                       str(normal_in_training_set)+', '+
                       str(original_attack_in_training_set)+', '+
                       str(generated_attacks)+', '+
                       str(normal_in_test_set)+', '+
                       str(attacks_in_test_set)+', '+
                       str(CICIDS_test_X.shape[1])+', '+
                       ' REMOVE IDENTICAL ELEMENTS, '+
                       '{}, {}, {}, {}, {:3f}, {:3f} \n'.format(tp, tn, fp, fn, accuracy, mcc))
cicids_competitors.flush()

In [ ]:
contamination=0.1

total=int(CICIDS_train_X_no_attack.shape[0]*contamination)

if(total > X_ood.shape[0]):
    print("contamination is larger than the available attacks")
    sys.exit(0)

    
idx = np.random.randint(X_ood.shape[0], size=total)
attacks=X_ood[idx,:]
CICIDS_train_X__no_attacks_augmented=np.vstack((CICIDS_train_X_no_attack, X_ood))
CICIDS_train_Y__no_attacks_augmented=np.vstack((np.zeros((CICIDS_train_X_no_attack.shape[0],1)),
                                                 np.ones((X_ood.shape[0], 1))))

contamination=contamination/(1+contamination)
                                                 
contamination

In [ ]:
isol_1=IsolationForest(n_estimators=300,
                     max_samples='auto',
                     contamination=contamination,
                     max_features=1.0,
                     bootstrap=False,
                     n_jobs=12,warm_start=False).fit(CICIDS_train_X__no_attacks_augmented)

predicted=isol_1.predict(CICIDS_test_X.to_numpy())

Y=CICIDS_test_Y.to_numpy().T[0]
predicted=np.where(predicted==-1, 1, 0)
accuracy=accuracy_score(Y, predicted)
mcc=matthews_corrcoef(Y, predicted)
tn, fp, fn, tp= confusion_matrix(Y, predicted).ravel()
tp, tn, fp, fn, accuracy, mcc

In [ ]:
dataset='cicids'
normal_in_training_set=np.unique(CICIDS_train_X_no_attack.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]
generated_attacks=X_ood.shape[0]
cicids_competitors.write(dataset+', '+
                       'TabGAN, '+
                       'Isolation Forest, '+
                       '0.6--0.4, '+
                       str(normal_in_training_set)+', '+
                       str(original_attack_in_training_set)+', '+
                       str(generated_attacks)+', '+
                       str(normal_in_test_set)+', '+
                       str(attacks_in_test_set)+', '+
                       str(CICIDS_test_X.shape[1])+', '+
                       ' REMOVE IDENTICAL ELEMENTS, '+
                       '{}, {}, {}, {}, {:3f}, {:3f} \n'.format(tp, tn, fp, fn, accuracy, mcc))
cicids_competitors.flush()

In [ ]:
clf = CBLOF(n_jobs=12, contamination=contamination)#, n_clusters=8
clf.fit(CICIDS_train_X__no_attacks_augmented)
predicted = clf.predict(CICIDS_test_X.to_numpy())  # outlier labels (0 or 1)
Y=CICIDS_test_Y.to_numpy().T[0]
accuracy=accuracy_score(Y, predicted)
mcc=matthews_corrcoef(Y, predicted)
tn, fp, fn, tp= confusion_matrix(Y, predicted).ravel()
tp, tn, fp, fn, accuracy, mcc

In [ ]:
dataset='cicids'
normal_in_training_set=np.unique(CICIDS_train_X_no_attack.shape[0])
original_attack_in_training_set=0
normal_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][0]
attacks_in_test_set=np.unique(CICIDS_test_Y, return_counts=True)[1][1]
generated_attacks=X_ood.shape[0]
cicids_competitors.write(dataset+', '+
                       'TabGAN, '+
                       'CBLOF, '+
                       '0.6--0.4, '+
                       str(normal_in_training_set)+', '+
                       str(original_attack_in_training_set)+', '+
                       str(generated_attacks)+', '+
                       str(normal_in_test_set)+', '+
                       str(attacks_in_test_set)+', '+
                       str(CICIDS_test_X.shape[1])+', '+
                       ' REMOVE IDENTICAL ELEMENTS, '+
                       '{}, {}, {}, {}, {:3f}, {:3f} \n'.format(tp, tn, fp, fn, accuracy, mcc))
cicids_competitors.flush()

In [ ]:
#saving generated attacks
np.save('/home/notebook/cicids/Tab_GAN.npy',X_ood)